In [1]:
from src.get_from_mongo import get_data
from sklearn.model_selection import train_test_split
from pymongo import MongoClient
import numpy as np
import tensorflow as tf
import pickle

In [2]:
id_from_char = {
    'CAPTAIN_FALCON' : 1 ,
    'DONKEY_KONG'    : 2 ,
    'FOX'            : 3 ,
    'GAME_AND_WATCH' : 4 ,
    'KIRBY'          : 5 ,
    'BOWSER'         : 6 ,
    'LINK'           : 7 ,
    'LUIGI'          : 8 ,
    'MARIO'          : 9 ,
    'MARTH'          : 10 ,
    'MEWTWO'         : 11 ,
    'NESS'           : 12 ,
    'PEACH'          : 13 ,
    'PIKACHU'        : 14 ,
    'ICE_CLIMBERS'   : 15 ,
    'JIGGLYPUFF'     : 16 ,
    'SAMUS'          : 17 ,
    'YOSHI'          : 18 ,
    'ZELDA'          : 19 ,
    'SHEIK'          : 20 ,
    'FALCO'          : 21 ,
    'YOUNG_LINK'     : 22 ,
    'DR_MARIO'       : 23 ,
    'ROY'            : 24 ,
    'PICHU'          : 25 ,
    'GANONDORF'      : 26 ,
}

char_from_id = {v:k for k, v in id_from_char.items()}

In [3]:
database_name = 'slippi'
collection_name = 'melee_clips_30s'

# Connect to the hosted MongoDB instance
client = MongoClient('localhost', 27017)
db = client[database_name]
collection = db[collection_name]

In [20]:
def data_generator(clip_collection=collection, # collection containing clips
                   batch_size = 100,
                   skip=None):

        cur = clip_collection.find()
        
        if skip:
            cur.skip(skip)
            
        while cur.alive:
            
            xi = []
            yi = []
            
            for _ in range(batch_size):
                clip = next(cur)
                xi.append(pickle.loads(clip['istream']).toarray())
                yi.append(id_from_char[clip['character']])

            Xi = np.stack(xi, axis=0)
            Yi = tf.one_hot(yi, 26)

            yield Xi, Yi

In [5]:
def get_test_data(clip_collection=collection, # collection containing clips
                  size = 100):
    bytestreams, characters = get_data('slippi', 
                                       'melee_public_slp_dataset',
                                       get=6000)

In [6]:
# gen = data_generator(I_train, batch_size=10)

In [7]:
# Xi, Yi = next(gen)
# for Y in Yi:
#     print(char_from_id[np.argmax(Y)])

In [8]:
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, InputLayer
from tensorflow.keras.layers import Conv1D, MaxPooling1D

m = keras.metrics.TopKCategoricalAccuracy(k=8)

In [9]:
model = Sequential()

# first conv layer
model.add(Conv1D(100, #num of features extracted from istream
                 60, #number of frames filter can see at once
                 activation='relu'))

model.add(MaxPooling1D(pool_size=5))

model.add(Conv1D(80,
                 30,
                 activation='relu'))

model.add(MaxPooling1D(pool_size=5))

model.add(Conv1D(60,
                 15,
                 activation='relu'))

model.add(MaxPooling1D(pool_size=5))

model.add(Flatten())

model.add(Dense(40, activation='relu'))

# final output layer
model.add(Dense(26, activation='softmax'))
                
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'categorical_accuracy', 'top_k_categorical_accuracy'])

In [16]:
data = data_generator(batch_size=100)

In [11]:
# during fit process watch train and test error simultaneously
model.fit(data, epochs=4, steps_per_epoch=500, verbose=1)

score = model.evaluate(data, steps=100, verbose=0)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')
print(f'Test categorical accuracy: {round(score[1]*100)}%')
print(f'Test test top 5 categorical accuracy: {round(score[1]*100)}%')  # this is the one we care about

Epoch 1/4
500/500 [==============================] - 467s 933ms/step - loss: 1.7437 - accuracy: 0.4396 - categorical_accuracy: 0.4396 - top_k_categorical_accuracy: 0.8144
Epoch 2/4
500/500 [==============================] - 474s 948ms/step - loss: 1.0263 - accuracy: 0.7029 - categorical_accuracy: 0.7029 - top_k_categorical_accuracy: 0.8916
Epoch 3/4
500/500 [==============================] - 473s 945ms/step - loss: 0.8220 - accuracy: 0.7625 - categorical_accuracy: 0.7625 - top_k_categorical_accuracy: 0.9124
Epoch 4/4
500/500 [==============================] - 470s 940ms/step - loss: 7.6756 - accuracy: 0.5788 - categorical_accuracy: 0.5788 - top_k_categorical_accuracy: 0.8294

Test score: 2.012
Test accuracy: 35%
Test categorical accuracy: 35%
Test test top 5 categorical accuracy: 35%


In [12]:
# during fit process watch train and test error simultaneously
model.fit(data, epochs=4, steps_per_epoch=500, verbose=1)

score = model.evaluate(data, steps=100, verbose=0)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')
print(f'Test categorical accuracy: {round(score[1]*100)}%')
print(f'Test test top 5 categorical accuracy: {round(score[1]*100)}%')  # this is the one we care about

Epoch 1/4
500/500 [==============================] - 473s 945ms/step - loss: 1.7324 - accuracy: 0.3828 - categorical_accuracy: 0.3828 - top_k_categorical_accuracy: 0.8295
Epoch 2/4
500/500 [==============================] - 466s 931ms/step - loss: 1.6304 - accuracy: 0.3995 - categorical_accuracy: 0.3995 - top_k_categorical_accuracy: 0.8484
Epoch 3/4
500/500 [==============================] - 466s 931ms/step - loss: 1.6050 - accuracy: 0.4262 - categorical_accuracy: 0.4262 - top_k_categorical_accuracy: 0.8433
Epoch 4/4
500/500 [==============================] - 466s 933ms/step - loss: 1.5010 - accuracy: 0.4706 - categorical_accuracy: 0.4706 - top_k_categorical_accuracy: 0.8593

Test score: 1.468
Test accuracy: 48%
Test categorical accuracy: 48%
Test test top 5 categorical accuracy: 48%


In [13]:
# during fit process watch train and test error simultaneously
model.fit(data, epochs=4, steps_per_epoch=500, verbose=1)

score = model.evaluate(data, steps=100, verbose=0)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')
print(f'Test categorical accuracy: {round(score[1]*100)}%')
print(f'Test test top 5 categorical accuracy: {round(score[1]*100)}%')  # this is the one we care about

Epoch 1/4
500/500 [==============================] - 467s 935ms/step - loss: 1.3451 - accuracy: 0.5480 - categorical_accuracy: 0.5480 - top_k_categorical_accuracy: 0.8740
Epoch 2/4
500/500 [==============================] - 467s 933ms/step - loss: 8.5794 - accuracy: 0.6436 - categorical_accuracy: 0.6436 - top_k_categorical_accuracy: 0.8746
Epoch 3/4
500/500 [==============================] - 466s 931ms/step - loss: 486.5970 - accuracy: 0.4509 - categorical_accuracy: 0.4509 - top_k_categorical_accuracy: 0.8940
Epoch 4/4
500/500 [==============================] - 466s 932ms/step - loss: 24708.1855 - accuracy: 0.3475 - categorical_accuracy: 0.3475 - top_k_categorical_accuracy: 0.9815

Test score: 63195.211
Test accuracy: 40%
Test categorical accuracy: 40%
Test test top 5 categorical accuracy: 40%


In [14]:
# during fit process watch train and test error simultaneously
model.fit(data, epochs=4, steps_per_epoch=500, verbose=1)

score = model.evaluate(data, steps=100, verbose=0)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')
print(f'Test categorical accuracy: {round(score[1]*100)}%')
print(f'Test test top 5 categorical accuracy: {round(score[1]*100)}%')  # this is the one we care about

Epoch 1/4
500/500 [==============================] - 468s 936ms/step - loss: 187739.4219 - accuracy: 0.3043 - categorical_accuracy: 0.3043 - top_k_categorical_accuracy: 0.9993
Epoch 2/4
500/500 [==============================] - 467s 935ms/step - loss: 493421.2500 - accuracy: 0.3049 - categorical_accuracy: 0.3049 - top_k_categorical_accuracy: 1.0000
Epoch 3/4
500/500 [==============================] - 467s 934ms/step - loss: 2706872.5000 - accuracy: 0.2396 - categorical_accuracy: 0.2396 - top_k_categorical_accuracy: 1.0000
Epoch 4/4
500/500 [==============================] - 467s 934ms/step - loss: 5970896.5000 - accuracy: 0.2341 - categorical_accuracy: 0.2341 - top_k_categorical_accuracy: 1.0000

Test score: 11198802.0
Test accuracy: 22%
Test categorical accuracy: 22%
Test test top 5 categorical accuracy: 22%


In [17]:
# during fit process watch train and test error simultaneously
model.fit(data, epochs=1, steps_per_epoch=500, verbose=1)

score = model.evaluate(data, steps=100, verbose=0)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')
print(f'Test categorical accuracy: {round(score[1]*100)}%')
print(f'Test test top 5 categorical accuracy: {round(score[1]*100)}%')  # this is the one we care about

500/500 [==============================] - 471s 942ms/step - loss: 10881439.0000 - accuracy: 0.2251 - categorical_accuracy: 0.2251 - top_k_categorical_accuracy: 1.0000

Test score: 7828629.0
Test accuracy: 27%
Test categorical accuracy: 27%
Test test top 5 categorical accuracy: 27%


In [18]:
def get_conf_matrix(labels_as_id, predictions_as_id):
    conf_matrix = np.zeros((27,27))
    for i_real, i_pred in zip(labels_as_id, predictions_as_id):
        conf_matrix[i_real, i_pred] += 1
    return conf_matrix

In [51]:
X_test, Y_test = next(data_generator(batch_size=5000, skip = 801000))

In [52]:
pred = np.argmax(model.predict(X_test), axis = 1)
y_test = np.argmax(Y_test, axis = 1)

In [53]:
conf_matrix = get_conf_matrix(y_test, pred)
char_id = 0

In [55]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('\nTest score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')
print(f'Test categorical accuracy: {round(score[1]*100)}%')
print(f'Test test top 5 categorical accuracy: {round(score[1]*100)}%') 


Test score: 7422518.0
Test accuracy: 26%
Test categorical accuracy: 26%
Test test top 5 categorical accuracy: 26%


In [61]:
char_id += 1
row = conf_matrix[char_id, :]
sorted_row_indices = np.argsort(row)[::-1]
print(f'{char_from_id[char_id]} : prediction frequencies\n-----------------')
for i in sorted_row_indices:
    if i > 0:
        print(f'{char_from_id[i]} : {row[i]}')

BOWSER : prediction frequencies
-----------------
CAPTAIN_FALCON : 6.0
GANONDORF : 0.0
NESS : 0.0
DONKEY_KONG : 0.0
FOX : 0.0
GAME_AND_WATCH : 0.0
KIRBY : 0.0
BOWSER : 0.0
LINK : 0.0
LUIGI : 0.0
MARIO : 0.0
MARTH : 0.0
MEWTWO : 0.0
PEACH : 0.0
PICHU : 0.0
PIKACHU : 0.0
ICE_CLIMBERS : 0.0
JIGGLYPUFF : 0.0
SAMUS : 0.0
YOSHI : 0.0
ZELDA : 0.0
SHEIK : 0.0
FALCO : 0.0
YOUNG_LINK : 0.0
DR_MARIO : 0.0
ROY : 0.0


In [ ]:
def test(i, ii):
    character = characters[i]
    istream = pickle.loads(bytestreams[i])

    y_test = []
    x_test = []
    f=0

    while f+F < istream.shape[0]:
        y_test.append(id_from_char[character])
        x_test.append(istream[f:f+F])
        f += F
        
    try:
        d = x_test[ii].toarray()
    except IndexError:
        test(i, ii-1)
        return
    
    pred = char_from_id[np.argmax(model.predict(d.reshape(1,d.shape[0],d.shape[1])))]


    print(f'''
    Actual Character:
    ---------------
    {char_from_id[y_test[ii]]}
    ''')

    print(f'''
    Detected Character:
    ---------------
    {pred}
    ''')

In [ ]:
# i = 602 # use this to check different entries
i = 630 # use this to check different entries
ii = 3 # use this to look at different clips from that entry

In [ ]:
print(characters[i], '\n', f'i={i}')

In [ ]:
test(i, ii)
i+=1

In [ ]:
i-=1